# ECEN760 Assignment-2: Text Classification using Multinomial Naive Bayes

   **Name:**  "Anil B Murthy"                      
   
   **UIN: **  "525006147"

## Instructions

In this assignment, you will write a python program to classify texts using a Multinomial Naive Bayes classifier. 

* Since this is your first programming assignment using python, you're provided with the sketch of the program. You only need to complete the cells that says " # Write your code here ". 

* Feel free to organize your code into multiple cells for better readability. 
* You may also create sub-functions/ definitions.
* You may also import more standard libraries in addition to the libraries that are already imported in the preprocessing part.


### Dataset

**20 Newsgroups** ( http://qwone.com/~jason/20Newsgroups/ )

The dataset has approximately 18,000 Newsgroups documents on 20 topics split in two subsets: one for training (or development) and the other one for testing (or for performance evaluation).

For this assignment, we will only use documents from 3 categories ('talk.religion.misc','comp.graphics','sci.space').

We recommend you to take advantage of the scikit-learn library to import and process the dataset. More information can be found in the following link:

   http://scikit-learn.org/stable/datasets/twenty_newsgroups.html
   
   

### 1. Pre-processing

In this step, we'll download the dataset using the scikit-learn library. The sample program to import the library is given below.

This involves two steps:

    1) Fetch dataset corresponding to the three following categories:

    *  'talk.religion.misc'
    *  'comp.graphics'
    *  'sci.space'
    
    2) Remove stop words* and create count vectors for the train and test datasets. 
    

    *Stop words in this context refer to words that occur very frequently and thus contain little information about the type of the article itself.  For e.g., 'a','and','the' etc. See https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/feature_extraction/stop_words.py for the list of stop words used in scikit when 'english' is given as input.
        

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
import numpy as np
from scipy.sparse import find

# Feel free to import any standard libraries that you may need to complete the program.

**Step1: ** Fetch the dataset for the three aforementioned categories using scikit-learn library.

In [2]:
categories = ['talk.religion.misc','comp.graphics','sci.space']

num_categories = len(categories)

#Loading training data

data_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

#Loading testing data

data_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

# Loading the class labels for training and testing data

y_train, y_test = data_train.target, data_test.target

In [3]:
# Total number of documents in train and test datasets

num_train = len(data_train.target)
num_test = len(data_test.target)

print("Dataset contatins \n "
       +str(num_train)+" train documents, \n "
       + str(num_test) + " test documents." )

Dataset contatins 
 1554 train documents, 
 1034 test documents.


Now, let's print a sample document to understand the dataset better.

Fill in the cell below to print contents of the first document from "train" subset. Also, print its corresponding class label name(category).

**Hint:** Use "data_train.data" variable

In [4]:
print("Sample train Document:\n")
print(data_train.data[0])
print("\n\n The above document belongs to the following category:\t")
print(categories[data_train.target[0]])

Sample train Document:

From: nicho@vnet.IBM.COM (Greg Stewart-Nicholls)
Subject: Re: Biosphere II
Reply-To: nicho@vnet.ibm.com
Disclaimer: This posting represents the poster's views, not those of IBM
News-Software: UReply 3.1
X-X-From: nicho@vnet.ibm.com
            <1q1kia$gg8@access.digex.net>
Lines: 18

In <1q1kia$gg8@access.digex.net> Pat writes:
>In article <19930408.043740.516@almaden.ibm.com> nicho@vnet.ibm.com writes:
>>In <1q09ud$ji0@access.digex.net> Pat writes:
>>>Why is everyone being so critical of B2?
>> Because it's bogus science, promoted as 'real' science.
>It seems to me, that it's sorta a large engineering project more
>then a science project.
  Bingo.
>B2 is not bench science,  but rather a large scale attempt to
>re-create a series of micro-ecologies.   what's so eveil about this?
 Nothing evil at all. There's no actual harm in what they're doing, only
how they represent it.

 -----------------------------------------------------------------
 .sig files are like s

**Step2:** Remove stop words and create count vectors for the train and test datasets.

   We use the CountVectorizer method to extract features (counts for each word). Note that words from both training and testing data are needed to build the count table.
   
   *Documentation:*  http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [5]:
vectorizer = CountVectorizer(stop_words='english')
vectorizer.fit(data_train.data + data_test.data)
x_train = vectorizer.transform(data_train.data)
x_test = vectorizer.transform(data_test.data)
vocab_length = len(vectorizer.vocabulary_)
print("Total number of words in vocabulary = "+str(vocab_length))
#print(x_test.getrow(1))

Total number of words in vocabulary = 37830


# 2. Building the classifier

Now, let's build a Multinomial Naive Bayes classifier that takes feature vector from the test data as input and classifies as one of the three classes ('talk.religion.misc','comp.graphics','sci.space').

Complete the training function MultiNB_train() in the cell below to train a Multiomial Naive Bayes classifier that takes "x_train","y_train","alpha" as inputs and returns the likelihood probability matrix "theta" and the prior distribution  "prior" on the document category.

"prior" is a vector of length equal to num_categories where the $i$-th element is defined as
$$ prior (i) = \frac{\text{ # of train documents with category i}}{\text{Total number of train documets}} $$

"theta" ($\theta$) is the  matrix with the $(c,i)$th element defined by

 $$ \theta(c,i) = P(w_i/c) =  \frac{N_{ci} + \alpha }{N_c + |V| \alpha}$$
 
 where,
 * $P(w_i/c)$ refers to the probability of seeing the $i$th word in the vocabulary given that class type is $c$.
 * $N_{ci}$ refers to the total number of times the word  $i$ appeared in the training documents of class type $c$.
 * $N_c$ is the total number of words in the documents of type $c$
    $$N_c = \sum_{d \in T[c]} N_{cd}$$
    where, $T[c]$ refers to the documents of type $c$.
 * $|V|$ is the size of the vocabulary.
 * $\alpha$ is the laplace smoothing parameter

***Note**: **Do NOT** use the scikit-learn's inbuilt function "MultinomialNB" . Write your own code to build the classifier. You may use standary libraries like "numpy","scipy" etc. to perform operations on matrices/arrays. 

Feel free to break your code into multiple functions or cells.

In [6]:
def MultiNB_train(x_train,y_train, alpha):
    #alpha = 0.01
    count = [0,0,0]
    for i in range(len(y_train)):
        if(y_train[i] == 0):
            count[0] = count[0] + 1
        elif(y_train[i] == 1):
            count[1] = count[1] + 1
        elif(y_train[i] == 2):
            count[2] = count[2] + 1
    prior = [(count[0]/float(num_train)),(count[1]/float(num_train)),(count[2]/float(num_train))]
    print(prior)
    print(x_train.shape)
    x_train_list = [((i, j), x_train[i,j]) for i, j in zip(*x_train.nonzero())] #converts CSR matrix format to List of tuples
    print(x_train_list[0])
    len_x_train = len(x_train_list)
    print(len_x_train)
    Nc = [0,0,0]
    N0i = [0] * vocab_length
    N1i = [0] * vocab_length
    N2i = [0] * vocab_length

    for i in range(len_x_train):
        category = data_train.target[x_train_list[i][0][0]]
        if(category == 0):
            Nc[0] = Nc[0] + x_train_list[i][1]
            N0i[x_train_list[i][0][1]] += x_train_list[i][1]
        elif(category == 1):
            Nc[1] = Nc[1] + x_train_list[i][1]
            N1i[x_train_list[i][0][1]] += x_train_list[i][1]
        elif(category == 2):
            Nc[2] = Nc[2] + x_train_list[i][1]
            N2i[x_train_list[i][0][1]] += x_train_list[i][1]

    theta0 = [((x + alpha)/float(Nc[0] + (vocab_length*alpha))) for x in N0i]
    theta1 = [((x + alpha)/float(Nc[1] + (vocab_length*alpha))) for x in N1i]
    theta2 = [((x + alpha)/float(Nc[2] + (vocab_length*alpha))) for x in N2i]

    theta = [theta0, theta1, theta2]
    print(len(theta0))
    print(len(theta1))
    print(len(theta2))
    print(len(theta))
    print(theta[0][9])

    return(theta, prior)

Now, let us train the model to learn the likelihood parameters $\theta$

In [7]:
theta,prior = MultiNB_train(x_train,y_train,alpha = 17)

[0.3758043758043758, 0.3815958815958816, 0.2425997425997426]
(1554, 37830)
((0, 192), 1)
175894
37830
37830
37830
3
2.33655778307e-05


Complete the classifier function MultiNB_classify() below that takes in features of one test sample (one row from x_test) and returns the predicted class "pred_class" $\in \{0,1,2\}$. 

In [8]:
def MultiNB_classify(x_test_sample, theta, prior):
    category0 = 0
    category1 = 0
    category2 = 0
    sample_list = [((i, j), x_test_sample[i,j]) for i, j in zip(*x_test_sample.nonzero())]
    sample_len = len(sample_list)
    for i in range(sample_len):
        word_no = sample_list[i][0][1]
        word_count = sample_list[i][1]
        max_prob = max(theta[0][word_no], theta[1][word_no], theta[2][word_no])
        if(max_prob == theta[0][word_no]):
            category0 += 1
        elif(max_prob == theta[1][word_no]):
            category1 += 1
        else:
            category2 += 1
    possible_class = max(category0, category1, category2)
    if(possible_class == category0):
        pred_class = 0
    elif(possible_class == category1):
        pred_class = 1
    else:
        pred_class = 2 
    
    return  pred_class

   

Let us test our classifier on the first sample of testing dataset.

In [9]:
pred_class = MultiNB_classify(x_test.getrow(0),theta, prior)

print("predicted class:" + str(pred_class))
print("actual class:" + str(y_test[0]))

predicted class:0
actual class:0


# 3. Evaluating the classifier

The following code below runs your classifier on every data sample from the testing dataset and stored them in "y_pred".

In [10]:
y_pred = []
for i in range(num_test):
    pred_class = MultiNB_classify(x_test.getrow(i),theta=theta, prior= prior)
    y_pred.append(pred_class)

The following cell evaluates your result by comparing it with the test labels.

In [11]:
score = metrics.accuracy_score(y_test,y_pred)

print("accuracy: %0.3f" % score)
print(metrics.classification_report(y_test,y_pred))

accuracy: 0.916
             precision    recall  f1-score   support

          0       0.97      0.87      0.92       389
          1       0.92      0.93      0.93       394
          2       0.85      0.95      0.90       251

avg / total       0.92      0.92      0.92      1034



Find the classification error (1-score) over the test set for various values of the smoothing parameter α and by trial and error find a good value of α.

In [12]:
error = 1 - score
print("Error value = "+str(error))

Error value = 0.0841392649903


Least Error Obtained = 0.0841392649903

--------------------------------------------------------------------------------------------------------------------------------

From Trail & Error,
the best values of alpha observed are between 16.25 to 17. 